<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import backtesting
from backtesting import Backtest
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting.test import SMA
import pandas_datareader.data as dtr
import requests_cache
import datetime
expire_after = datetime.timedelta(days = 7)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)

tickers = ["LNTA.ME","MGNT.ME","PIKK.ME",
           "MTSS.ME","MGTS.ME","RTKM.ME",
           "ROSN.ME","GAZP.ME","NVTK.ME","LKOH.ME","TATN.ME","SNGS.ME","SIBN.ME",
           "PLZL.ME","NLMK.ME","CHMF.ME","POLY.ME","RUAL.ME","ALRS.ME",
           "SBER.ME","VTBR.ME","MOEX.ME","CBOM.ME","YNDX.ME","CBOM.ME","ROSB.ME",
           "HYDR.ME","RSTI.ME","FEES.ME"
          ]

df = pd.DataFrame()

for ticker in tickers:
    try:
        df_ = dtr.DataReader(ticker, data_source="yahoo" , session=session, retry_count=1)
        df_.insert(0, "ticker", ticker)
    except:
        print(f"ticker '{ticker}' failed")
        pass
    df = df.append(df_)

GAZP = df[df.ticker == "GAZP.ME"]
GAZP.head()

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

,ticker,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
2015-12-17,GAZP.ME,138.690002,134.649994,134.649994,137.899994,45419720.0,137.899994
2015-12-18,GAZP.ME,137.389999,132.250000,137.289993,132.250000,38544860.0,132.250000
2015-12-21,GAZP.ME,133.940002,131.000000,132.000000,131.500000,28393100.0,131.500000
2015-12-22,GAZP.ME,133.539993,131.809998,132.000000,133.199997,20022420.0,133.199997
2015-12-23,GAZP.ME,135.300003,133.350006,133.550003,135.300003,26760910.0,135.300003


In [37]:
class SmaCross(SignalStrategy, TrailingStrategy):
    
    n1 = 10
    n2 = 30
    
    def init(self):
        super().init()
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        # Upwards/long only
        signal = signal.replace(-1, 0)
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

ret = 0
for m in [i/10 for i in range(1,10)]:
    bt = Backtest(GAZP, SmaCross, commission = .002, margin =m)
    stats=bt.run()
    if stats['Sharpe Ratio'] > ret:
        ret = stats['Sharpe Ratio']
        margin = m 
margin, ret

(0.9, 0.5048275058688446)

In [40]:
bt = Backtest(GAZP, SmaCross, commission = .002, margin=.5)
stats = bt.optimize(n1 = range(10,30,5),
                    n2 = range(20,70,5),
                    maximize = 'Sharpe Ratio',
                    constraint = lambda param: param.n1 < param.n2
                   )
stats

Start                     2015-12-17 00:00:00
End                       2020-12-14 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                     29.7686
Equity Final [$]                      36393.6
Equity Peak [$]                       36393.6
Return [%]                            263.936
Buy & Hold Return [%]                 44.6773
Return (Ann.) [%]                     29.6677
Volatility (Ann.) [%]                 31.9495
Sharpe Ratio                         0.928579
Sortino Ratio                         2.66769
Calmar Ratio                          1.41188
Max. Drawdown [%]                    -21.0129
Avg. Drawdown [%]                    -5.02658
Max. Drawdown Duration      372 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                   15
Win Rate [%]                          73.3333
Best Trade [%]                        22.9381
Worst Trade [%]                      -5.98213
Avg. Trade [%]                    